In [ ]:
# (Colab 등에서 한 번만)
# !pip install networkx openai tqdm

import os
import json
import time
from typing import List, Dict

import networkx as nx
from tqdm.auto import tqdm
from openai import OpenAI
import openai  # 예외 타입용 (APITimeoutError, RateLimitError)

# 🔑 OpenAI 클라이언트 (환경변수 OPENAI_API_KEY 사용)
client = OpenAI(api_key = "your key")
DATA_DIR = "Amazon_products"

# ====== 설정값 ======
DOCS_PER_PATH = 5                  # path당 생성할 문서 수 (최대 n_docs)

MAX_CALLS = 1000   # 전체 API 호출 예산
call_count = 0     # 전역 카운터


In [ ]:
def read_taxonomy(taxonomy_file: str, id2label: Dict[str, str]) -> nx.DiGraph:
    """
    label_hierarchy.txt 를 읽어서 root 가 추가된 DiGraph 를 만든다.
    파일 포맷: parent_id \t child_id
    """
    graph = nx.DiGraph()
    with open(taxonomy_file, "r", encoding="utf-8") as f:
        for line in f:
            if line == "\n":
                continue
            p_id, c_id = line.strip().split("\t")
            source_node = id2label[p_id]
            target_node = id2label[c_id]

            graph.add_node(source_node)
            graph.add_node(target_node)
            graph.add_edge(source_node, target_node)

    # 부모 없는 노드들에 대해 root 추가
    no_parents = [node for node in graph.nodes if graph.in_degree(node) == 0]
    if len(no_parents) != 0:
        graph.add_node("root")
        for np in no_parents:
            graph.add_edge("root", np)

    return graph


def get_all_paths(graph: nx.DiGraph, start_node: str = "root") -> List[str]:
    """
    root -> leaf 모든 경로를 "\\" 구분자로 이어붙인 문자열 리스트로 반환
    예: "Electronics\\Camera"
    """
    leaf_nodes = [node for node in graph.nodes() if graph.out_degree(node) == 0]

    all_paths = []
    for leaf_node in leaf_nodes:
        paths = list(nx.all_simple_paths(graph, source=start_node, target=leaf_node))
        for path in paths:
            # "root"는 문자열에서 제거
            if "root" in path:
                path.pop(path.index("root"))
            all_paths.append("\\".join(path))
    return all_paths


In [ ]:
def extract_json_list(content: str):
    """
    모델이 준 content에서 [ ... ] 로 보이는 JSON array 부분만 뽑아서 json.loads 하는 함수.
    """
    content = content.strip()

    # 1) 코드 블록(``` ... ```) 안에 JSON이 들어있는 경우 먼저 처리
    if "```" in content:
        parts = content.split("```")
        # parts 중에서 [ 로 시작하고 ] 로 끝나는 부분을 찾아본다
        for part in parts:
            part = part.strip()
            if part.startswith("[") and part.endswith("]"):
                return json.loads(part)

    # 2) 코드 블록이 아니고, 그냥 앞뒤에 텍스트 + [ ... ]만 있는 경우
    start = content.find("[")
    end = content.rfind("]")
    if start != -1 and end != -1 and start < end:
        snippet = content[start : end + 1]
        return json.loads(snippet)

    # 여기까지 안 걸리면 그냥 실패
    raise json.JSONDecodeError("Could not extract JSON array", content, 0)

def _build_messages_for_path(path: str, dataset: str):
    """
    path: 예) 'Electronics\\Camera'
    dataset: 'Amazon-531' 또는 'DBPedia-298'
    """
    backslash_char = "\\"
    leaf = path.split(backslash_char)[-1]

    system_prompt = (
            "You are an Amazon product reviewer. "
            "You write detailed, realistic product reviews."
        )
    user_prompt = f"""\
Suppose you are an Amazon Reviewer, please generate 5 various and reliable passages following the requirements below:
1. Must generate reviews following the themes of the taxonomy path: {path} but **without** explicitly mentioning "{path}".
2. Each review must be longer than 100 words.
3. The writing style and format of the text should be a product review.
4. Keep the generated texts diverse, specific, and consistent with the given taxonomy path.
You should focus on "{leaf}" as the main topic of the documents.

Your output MUST be a valid JSON list of objects like:
[{{"doc": "text1"}}, {{"doc": "text2"}}, ...]
"""
    min_len = 100


    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    return messages, min_len


def generate_pseudo_docs_for_path(
    path: str,
    dataset: str,
    n_docs: int = 5,
    max_retries: int = 5,
):
    """
    하나의 taxonomy path 에 대해 pseudo documents를 생성해서 리스트로 리턴.
    - path: 'Electronics\\Camera' 같은 형태
    - dataset: 'Amazon-531' 또는 'DBPedia-298'
    - n_docs: 최종적으로 받고 싶은 문서 개수 (최대)
    """
    global call_count

    docs = []
    retries = 0

    while len(docs) < n_docs and retries < max_retries:
        if call_count >= MAX_CALLS:
            print("[WARN] MAX_CALLS reached, stop generation.")
            break

        messages, min_len = _build_messages_for_path(path, dataset)

        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",  # 필요하면 다른 모델로 변경
                temperature=0.8,
                max_tokens=3000,
                messages=messages,
            )
            call_count += 1

            content = response.choices[0].message.content.strip()

            try:
                results_json = extract_json_list(content)
            except json.JSONDecodeError:
                print("[WARN] JSON decode error, retrying...")
                retries += 1
                continue

            for item in results_json:
                if "doc" not in item:
                    continue
                text = item["doc"].strip().replace("\n", " ")
                if len(text) < min_len:
                    continue
                # 원래 코드에서 걸러주던 이상 케이스
                if ":" in text or "=" in text:
                    continue

                docs.append(text)
                if len(docs) >= n_docs:
                    break

        except openai.APITimeoutError:
            print("[WARN] APITimeoutError, retrying...")
            retries += 1
            continue
        except openai.RateLimitError:
            print("[WARN] RateLimitError, sleep 10s...")
            time.sleep(10)
            retries += 1
            continue
        except Exception as e:
            print(f"[WARN] Unexpected error: {e}")
            retries += 1
            continue

        time.sleep(0.1)  # rate limit 완화용

    return docs


In [ ]:
# ===== 1) 라벨 파일 읽기 =====
labels_path = os.path.join(DATA_DIR, "classes.txt")
id2label = {}
label2id = {}

with open(labels_path, "r", encoding="utf-8") as f:
    for line in f.read().split("\n"):
        if line == "":
            continue
        id_str, label_name = line.split("\t")
        id2label[id_str] = label_name
        label2id[label_name] = id_str

print(f"# labels: {len(id2label)}")


# ===== 2) taxonomy & path 리스트 구하기 =====
taxo_path = os.path.join(DATA_DIR,"class_hierarchy.txt")
taxo = read_taxonomy(taxo_path, id2label)
paths = get_all_paths(taxo, start_node="root")
print(f"# taxonomy paths (root→leaf): {len(paths)}")


# ===== 3) 출력 경로 준비 =====
out_dir = os.path.join(DATA_DIR, "train")
os.makedirs(out_dir, exist_ok=True)

docs_txt_path = os.path.join(out_dir, "generated_docs.txt")
json_path = os.path.join(out_dir, "generated_doc2label.json")

id_docs = {}
doc_id_labels = {}
num_doc = 0

# ===== 4) path별로 pseudo docs 생성 + 파일에 저장 =====
with open(docs_txt_path, "w", encoding="utf-8") as doc_file:
    for path in tqdm(paths, desc="Generating pseudo docs per path"):
        if call_count >= MAX_CALLS:
            print("[INFO] Reached MAX_CALLS, stop generation loop.")
            break

        path_list = path.split("\\")
        leaf_label = path_list[-1]

        pseudo_docs = generate_pseudo_docs_for_path(
            path=path,
            dataset=DATASET,
            n_docs=DOCS_PER_PATH,
        )

        for doc in pseudo_docs:
            doc_id = str(num_doc)
            id_docs[doc_id] = doc

            local_doc_to_label = {
                "core_classes": label2id[leaf_label],
                "with ancestors": [label2id[label] for label in path_list],
            }
            doc_id_labels[doc_id] = local_doc_to_label

            safe_text = doc.replace("\n", " ")
            doc_file.write(f"{doc_id}\t{safe_text}\n")

            num_doc += 1

# ===== 5) doc2label json 저장 =====
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(doc_id_labels, f, ensure_ascii=False, indent=2)

print(f"Total pseudo docs generated: {num_doc}")
print(f"Docs file saved to: {docs_txt_path}")
print(f"Doc2label json saved to: {json_path}")
print(f"Total API calls used: {call_count}/{MAX_CALLS}")
#0.24 $
